# Interpolation examples

## Package loading and basic configurations

In [2]:
%load_ext autoreload
%autoreload 2

# load dependencies'
import pandas as pd
import geopandas as gpd

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig, Preprocessing, Visualiser

# create an initial but optional config and an api client
config = ECConfig()
preprocessing = Preprocessing()
visualiser = Visualiser()
track_api = TrackAPI(api_client=DownloadClient(config=config))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing pre-processing class
Initializing visualisation class


## Querying enviroCar Tracks

The following cell queries tracks from the enviroCar API. It defines a bbox for the area of Münster (Germany) and requests 50 tracks. The result is a GeoDataFrame, which is a geo-extended Pandas dataframe from the GeoPandas library. It contains all information of the track in a flat dataframe format including a specific geometry column.

In [4]:
bbox = BboxSelector([
    7.554130554199218, # min_x
    51.95590322041212, # min_y
    7.590351104736328, # max_x
    51.97874790276371  # max_y
])


# issue a query
track_df = track_api.get_tracks(bbox=bbox, num_results=10) # requesting 10 tracks inside the bbox

Removing outliers from the CO2 measurements

In [6]:
track_df = preprocessing.remove_outliers(track_df, 'CO2.value')

Checking what the original speeds look like.

In [17]:
visualiser.plot_point_values(track_df, "Speed.value")

Interpolating points for every second along the tracks.

In [16]:
interpolated_points_full = preprocessing.interpolate(track_df, step_type="seconds")
visualiser.plot_point_values(interpolated_points_full, "Speed.value")

Amount of points before interpolation (1952, 54)
Amount of points after interpolation (9846, 54)


Interpolating points for every meter of just one track since it is more computationally intensive. For that same reason the default is to calculate for every 10 meters.

In [18]:
single_track_id = track_df['track.id'].unique()[1]
single_track = track_df[track_df['track.id'] == single_track_id]

interpolated_one = preprocessing.interpolate(single_track, step_type="meters", step_pr=1)
visualiser.plot_point_values(interpolated_one, "CO2.value")

Amount of points before interpolation (138, 54)
Amount of points after interpolation (5658, 54)
